In [1]:
param_date = '20240726'

In [2]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/4.type_2'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['group_1.Apparel & Gifts.xlsx',
 'group_10.Electrical-Switch & Relay.xlsx',
 'group_11.Engine.xlsx',
 'group_12.Exhaust & Emission.xlsx',
 'group_13.Fuel & Air.xlsx',
 'group_14.Garage Equipment.xlsx',
 'group_15.Hardware.xlsx',
 'group_16.Heat & Air Conditioning.xlsx',
 'group_17.Hoses%2FLines & Clamps.xlsx',
 'group_18.Ignition.xlsx',
 'group_19.Interior.xlsx',
 'group_2.Belt Drive.xlsx',
 'group_20.Literature.xlsx',
 'group_21.Steering.xlsx',
 'group_22.Suspension.xlsx',
 'group_23.Transmission-Automatic.xlsx',
 'group_24.Transmission-Manual.xlsx',
 'group_25.Wheel.xlsx',
 'group_26.Wiper & Washer.xlsx',
 'group_3.Body & Lamp Assembly.xlsx',
 'group_4.Brake & Wheel Hub.xlsx',
 'group_5.Cooling System.xlsx',
 'group_6.Drivetrain.xlsx',
 'group_7.Electrical.xlsx',
 'group_8.Electrical-Bulb & Socket.xlsx',
 'group_9.Electrical-Connector.xlsx']

In [3]:
file = list_file[0]

file

'group_2.Belt Drive.xlsx'

In [4]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [5]:
input_ = pd.read_excel(f'./{param_date}/4.type_2/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['Group No'] = input_['Group No'].astype(int)
input_['Type No'] = input_['Type No'].astype(int)
input_['Manufacturer No'] = input_['Manufacturer No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Manufacturer'].tolist()

总数量：1


['GATES']

In [6]:
a = 0

a

0

In [7]:
crawler_status = 'error'

crawler_status

'error'

In [8]:
import requests

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import re

b = 0
while True:

    while True:
        b += 1
        print(b)
        try:
            resp = requests.get(input_.loc[a, 'Tab 2 Url'],
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))
            break
        except:
            continue

    if resp.status_code == 500:
        break
    elif resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'lxml')
        html = etree.HTML(str(soup))
        try:
            title = re.sub(r' +', ' ', html.xpath('//div[@id="nav_o[1]"]/div[@class="inner"]/div/div[@style="display: inline-block; padding: 0em 5em 0em 5em; "]/text()')[0]).strip()

            if html.xpath('//select[@id="manufacturer_partsearch_007"]/option[@selected]/@value')[0] == input_.loc[a, 'Manufacturer'] and html.xpath('//select[@id="partgroup_partsearch_007"]/option[@selected]/@value')[0] == input_.loc[a, 'Group'] and html.xpath('//select[@id="parttype_partsearch_007"]/option[@selected]/@value')[0] == input_.loc[a, 'Type Code'] and html.xpath('//select[@id="parttype_partsearch_007"]/option[@selected]/text()')[0] == input_.loc[a, 'Type'] or re.sub(r' +', ' ', f'''{input_.loc[a, 'Manufacturer']} {input_.loc[a, 'Type']} Part Numbers''').strip() == title:
                with open('./part_number.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
        except:
            continue

resp

1


<Response [200]>

In [9]:
list_json = html.xpath('//div[@id="navchildren[1]"]/div[contains(@id, "nav[")]/input[contains(@id, "jsn[")]/@value')

list_json

[]

In [10]:
import json

list_part_number = [json.loads(json_)['partnumber'] for json_ in list_json]

list_part_number

[]

In [11]:
list_url = [f'https://www.rockauto.com{href.strip()}' for href in html.xpath('//div[@id="navchildren[1]"]/div[contains(@id, "nav[")]/div[contains(@id, "nav_o[")]/div/table/tr/td[@class="nlabel nlbl-docolumns"]/a/@href')]

list_url

[]

In [12]:
len(list_part_number) == 0

True

In [13]:
df_temp = pd.DataFrame({'Group No': input_.loc[a, 'Group No'],
                        'Group': input_.loc[a, 'Group'],
                        'Type No': input_.loc[a, 'Type No'],
                        'Type': input_.loc[a, 'Type'],
                        'Type Code': input_.loc[a, 'Type Code'],
                        'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                        'Manufacturer': input_.loc[a, 'Manufacturer'],
                        'Tab 2 Url': input_.loc[a, 'Tab 2 Url'],
                        'SKU Count': len(list_part_number),
                        'Part No': [i+1 for i in range(len(list_part_number))],
                        'Part Number': list_part_number,
                        'Url': list_url})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Group No,Group,Type No,Type,Type Code,Manufacturer No,Manufacturer,Tab 2 Url,SKU Count,Part No,Part Number,Url


In [14]:
crawler_status = 'ok'

crawler_status

'ok'

In [15]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Group No': input_.loc[a, 'Group No'],
                             'Group': input_.loc[a, 'Group'],
                             'Type No': input_.loc[a, 'Type No'],
                             'Type': input_.loc[a, 'Type'],
                             'Type Code': input_.loc[a, 'Type Code'],
                             'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                             'Manufacturer': input_.loc[a, 'Manufacturer'],
                             'Tab 2 Url': input_.loc[a, 'Tab 2 Url']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [16]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Group No', 'Type No', 'Manufacturer No', 'Part No'],
                                                ascending=[True, True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part_number_1.xlsx', index=False)

output_correct

,Group No,Group,Type No,Type,Type Code,Manufacturer No,Manufacturer,Tab 2 Url,SKU Count,Part No,Part Number,Url


In [17]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Group No', 'Type No', 'Manufacturer No'],
                                            ascending=[True, True, True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_number_1_error.xlsx', index=False)

output_error

""
